In [1]:
%reset
from sympy import *
import numpy as np

In [12]:
# 1) Define plasticity model
a_p, a_ve, am = symbols('a_p, a_ve, am')

Diso = Matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, a_ve**-2, 0], [0, 0, 0, 0, a_ve**-2]])
iDpl = Matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, a_p**2,   0], [0, 0, 0, 0, a_p**2  ]])  # error 1
eta_ve, eta_vp, gdot, P, phi, Coh, half = symbols('eta_ve, eta_vp, gdot, P, fric, Coh, half')
a1, a2, a3 = symbols('a1, a2, a3')
Exx, Eyy, Ezz, Exy, Eyx = symbols('Exx, Eyy, Ezz, Exy, Eyx')
Eps = Matrix([[Exx], [Eyy], [Ezz], [Exy], [Eyx]])
Tau = eta_ve*Diso*Eps
display(Tau)
Txx, Tyy, Tzz, Txy, Tyx = symbols('Txx, Tyy, Tzz, Txy, Tyx')
Tau = Matrix([[Txx], [Tyy], [Tzz], [Txy], [Tyx]])
J2  =  Rational(1,2)*(Tau.transpose() * iDpl * Tau)[0]
Ty  = Coh + P*sin(phi)*am -  sin(phi)*( a1*Txx + a2*Tyy + a3*Tzz)*Rational(1,3)
# Ty  = Coh + P*sin(phi)*am
F   = sqrt( J2 ) - Ty 
display( F )

Matrix([
[        Exx*eta_ve],
[        Eyy*eta_ve],
[        Ezz*eta_ve],
[Exy*eta_ve/a_ve**2],
[Eyx*eta_ve/a_ve**2]])

-Coh - P*am*sin(fric) + (Txx*a1 + Tyy*a2 + Tzz*a3)*sin(fric)/3 + sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)

In [13]:
# 2) express corrected stress invariant
Eps_p =  gdot * Matrix([[F.diff(Txx)], [F.diff(Tyy)], [F.diff(Tzz)], [F.diff(Txy)], [F.diff(Tyx)] ])
# dFdTxx, dFdTyy, dFdTzz, dFdTxy, dFdTyx = symbols('dFdTxx, dFdTyy, dFdTzz, dFdTxy, dFdTyx')
# Eps_p =  gdot * Matrix([dFdTxx, dFdTyy, dFdTzz, dFdTxy, dFdTyx ])
display(Eps_p.subs(J2, 'J2') )
# Corrected stress
Tau_corr = Tau - 2*eta_ve*Diso*(Eps_p) # Tau = 2*eta_ve*Diso*(Eps - Eps_p)
# Simplify: factor out dev. stress components 
for i in range(5):
    Tau_corr[i] = collect(Tau_corr[i], Tau[i])
# Invariant
J2_corr  = Rational(1,2)*(Tau_corr.transpose() * iDpl * Tau_corr)[0]
# Simplify: collect out common factor
a = (1-(2*eta_ve*gdot)/sqrt(J2))**2 
J2_corr1 = collect(J2_corr, a).subs(J2, 'J2') 
display(J2_corr1)
julia_code(J2_corr1)

Matrix([
[gdot*(a1*sin(fric)/3 + Txx/(2*sqrt(J2)))],
[gdot*(a2*sin(fric)/3 + Tyy/(2*sqrt(J2)))],
[gdot*(a3*sin(fric)/3 + Tzz/(2*sqrt(J2)))],
[            Txy*a_p**2*gdot/(2*sqrt(J2))],
[            Tyx*a_p**2*gdot/(2*sqrt(J2))]])

Txy**2*a_p**2*(1 - a_p**2*eta_ve*gdot/(sqrt(J2)*a_ve**2))**2/2 + Tyx**2*a_p**2*(1 - a_p**2*eta_ve*gdot/(sqrt(J2)*a_ve**2))**2/2 + (Txx - 2*eta_ve*gdot*(a1*sin(fric)/3 + Txx/(2*sqrt(J2))))**2/2 + (Tyy - 2*eta_ve*gdot*(a2*sin(fric)/3 + Tyy/(2*sqrt(J2))))**2/2 + (Tzz - 2*eta_ve*gdot*(a3*sin(fric)/3 + Tzz/(2*sqrt(J2))))**2/2

'Txy .^ 2 .* a_p .^ 2 .* (1 - a_p .^ 2 .* eta_ve .* gdot ./ (sqrt(J2) .* a_ve .^ 2)) .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 .* (1 - a_p .^ 2 .* eta_ve .* gdot ./ (sqrt(J2) .* a_ve .^ 2)) .^ 2 / 2 + (Txx - 2 * eta_ve .* gdot .* (a1 .* sin(fric) / 3 + Txx ./ (2 * sqrt(J2)))) .^ 2 / 2 + (Tyy - 2 * eta_ve .* gdot .* (a2 .* sin(fric) / 3 + Tyy ./ (2 * sqrt(J2)))) .^ 2 / 2 + (Tzz - 2 * eta_ve .* gdot .* (a3 .* sin(fric) / 3 + Tzz ./ (2 * sqrt(J2)))) .^ 2 / 2'

In [4]:
J1  = sin(phi)/3*( a1*Txx + a2*Tyy + a3*Tzz)
J1c = sin(phi)/3*( a1*Tau_corr[0] + a2*Tau_corr[1] + a3*Tau_corr[2] )
display(J1c)
julia_code(J1c)

(Txx*a1*(-eta_ve*gdot/sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2) + 1) + Tyy*a2*(-eta_ve*gdot/sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2) + 1) + Tzz*a3*(-eta_ve*gdot/sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2) + 1))*sin(fric)/3

'(Txx .* a1 .* (-eta_ve .* gdot ./ sqrt(Txx .^ 2 / 2 + Txy .^ 2 .* a_p .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 / 2 + Tyy .^ 2 / 2 + Tzz .^ 2 / 2) + 1) + Tyy .* a2 .* (-eta_ve .* gdot ./ sqrt(Txx .^ 2 / 2 + Txy .^ 2 .* a_p .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 / 2 + Tyy .^ 2 / 2 + Tzz .^ 2 / 2) + 1) + Tzz .* a3 .* (-eta_ve .* gdot ./ sqrt(Txx .^ 2 / 2 + Txy .^ 2 .* a_p .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 / 2 + Tyy .^ 2 / 2 + Tzz .^ 2 / 2) + 1)) .* sin(fric) / 3'

In [5]:
Ji, Jii, P, fric, K, dt, dil      = symbols('Ji, Jii, P, fric, K, dt, dil')
am      = symbols('am')
Pc      = P + K*dt*gdot*sin(dil)*am
ap_n1   = 1 -          eta_ve * gdot / sqrt(Jii)
ap_s1   = 1 - a_p**2 * eta_ve * gdot / (sqrt(Jii)*a_ve**2)
# am      =  Rational(1,3) * (a1+a2+a3)

display(ap_n1.diff(gdot))
display(ap_s1.diff(gdot))
dapndgdot = ap_n1.diff(gdot)
dapsdgdot = dapndgdot*a_p**2/a_ve**2
# apres massage
ap_n    = Function('ap_n')(gdot)
ap_s    = Function('ap_s')(gdot)
J2_corr = Rational(1,2)*( (Txx**2 + Tyy**2 + Tzz**2)*ap_n**2 + a_p**2*(Txy**2 + Tyx**2)*ap_s**2)
J1_corr = ap_n*Ji
F       = sqrt(J2_corr) - eta_vp*gdot - Pc * am * sin(fric) + J1_corr
F.diff(gdot).subs(J2_corr,'J2_corr')


-eta_ve/sqrt(Jii)

-a_p**2*eta_ve/(sqrt(Jii)*a_ve**2)

Ji*Derivative(ap_n(gdot), gdot) - K*am**2*dt*sin(dil)*sin(fric) - eta_vp + (a_p**2*(Txy**2 + Tyx**2)*ap_s(gdot)*Derivative(ap_s(gdot), gdot)/2 + (Txx**2 + Tyy**2 + Tzz**2)*ap_n(gdot)*Derivative(ap_n(gdot), gdot)/2)/sqrt(J2_corr)

In [6]:
# try to find analytical solution
J2_corr = Rational(1,2)*( (Txx**2 + Tyy**2 + Tzz**2)*ap_n1**2 + a_p**2*(Txy**2 + Tyx**2)*ap_s1**2)
F       = sqrt(J2_corr) - 0*eta_vp*gdot - 0*Pc * am * sin(fric) + 0*J1_corr
display(F)
sol = solve(F,gdot)

sqrt(a_p**2*(1 - a_p**2*eta_ve*gdot/(sqrt(Jii)*a_ve**2))**2*(Txy**2 + Tyx**2)/2 + (1 - eta_ve*gdot/sqrt(Jii))**2*(Txx**2 + Tyy**2 + Tzz**2)/2)

In [7]:
julia_code(sol[1])

'a_ve .^ 2 .* (Jii .^ (5 // 2) .* (Txx .^ 2 .* a_ve .^ 2 + Txy .^ 2 .* a_p .^ 4 + Tyx .^ 2 .* a_p .^ 4 + Tyy .^ 2 .* a_ve .^ 2 + Tzz .^ 2 .* a_ve .^ 2) + sqrt(-Jii .^ 5 .* a_p .^ 2 .* (Txx .^ 2 .* Txy .^ 2 .* a_p .^ 4 - 2 * Txx .^ 2 .* Txy .^ 2 .* a_p .^ 2 .* a_ve .^ 2 + Txx .^ 2 .* Txy .^ 2 .* a_ve .^ 4 + Txx .^ 2 .* Tyx .^ 2 .* a_p .^ 4 - 2 * Txx .^ 2 .* Tyx .^ 2 .* a_p .^ 2 .* a_ve .^ 2 + Txx .^ 2 .* Tyx .^ 2 .* a_ve .^ 4 + Txy .^ 2 .* Tyy .^ 2 .* a_p .^ 4 - 2 * Txy .^ 2 .* Tyy .^ 2 .* a_p .^ 2 .* a_ve .^ 2 + Txy .^ 2 .* Tyy .^ 2 .* a_ve .^ 4 + Txy .^ 2 .* Tzz .^ 2 .* a_p .^ 4 - 2 * Txy .^ 2 .* Tzz .^ 2 .* a_p .^ 2 .* a_ve .^ 2 + Txy .^ 2 .* Tzz .^ 2 .* a_ve .^ 4 + Tyx .^ 2 .* Tyy .^ 2 .* a_p .^ 4 - 2 * Tyx .^ 2 .* Tyy .^ 2 .* a_p .^ 2 .* a_ve .^ 2 + Tyx .^ 2 .* Tyy .^ 2 .* a_ve .^ 4 + Tyx .^ 2 .* Tzz .^ 2 .* a_p .^ 4 - 2 * Tyx .^ 2 .* Tzz .^ 2 .* a_p .^ 2 .* a_ve .^ 2 + Tyx .^ 2 .* Tzz .^ 2 .* a_ve .^ 4))) ./ (Jii .^ 2 .* eta_ve .* (Txx .^ 2 .* a_ve .^ 4 + Txy .^ 2 .* a_p .^ 6 + Ty